In [1]:
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName('Genre Analysis').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/19 18:20:01 WARN Utils: Your hostname, Pulastyas-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 10.0.0.40 instead (on interface en0)
26/01/19 18:20:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/19 18:20:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
oscar = spark.read.options(header=True, inferSchema=True, sep='|')\
    .csv('data/input/oscar_nominees.csv')

oscar.show(truncate=False)

+----+----------------------------+---------------+-----------------------------------------------+------+---+
|year|category                    |nominee        |movie                                          |winner|id |
+----+----------------------------+---------------+-----------------------------------------------+------+---+
|2006|actress in a supporting role|Abigail Breslin|Little Miss Sunshine                           |false |1  |
|1984|actor in a supporting role  |Adolph Caesar  |A Soldier's Story                              |false |2  |
|2006|actress in a supporting role|Adriana Barraza|Babel                                          |false |3  |
|2002|actor in a leading role     |Adrien Brody   |The Pianist                                    |true  |4  |
|1942|actress in a supporting role|Agnes Moorehead|The Magnificent Ambersons                      |false |5  |
|1944|actress in a supporting role|Agnes Moorehead|Mrs. Parkington                                |false |6  |
|

In [18]:
nominee_info = spark.read.options(header=True, inferSchema=True)\
    .csv('data/input/nominee_information.csv')
    
nominee_info.filter(F.col('name') == 'Ruby Dee').show(truncate=False)

+--------+-------------+---------+----------+---+
|name    |amg_person_id|top_genre|birthday  |id |
+--------+-------------+---------+----------+---+
|Ruby Dee|P 18243      |Drama    |1924-10-27|234|
+--------+-------------+---------+----------+---+



In [30]:
winner = oscar.filter(F.col('winner') == 'true')\
    .groupBy('nominee').agg(F.count('*').alias('no_wins'))\
    .select(F.col('nominee'), F.col('no_wins'))
    
# max_wins = winner.groupBy().agg(F.max(F.col('no_wins')).alias('max_wins')).collect()[0][0]

max_wins = winner.groupBy().agg(F.max(F.col('no_wins')).alias('max_wins'))

# top_winner = winner.filter(F.col('no_wins') == max_wins)\
#     .orderBy(F.col('nominee')).limit(1)\
#     .join(nominee_info, on=nominee_info.name == F.col('nominee'), how='inner')\
#     .select(F.col('top_genre'))

top_winner = winner.crossJoin(max_wins)\
    .filter(F.col('no_wins') == F.col('max_wins'))\
    .orderBy(F.col('nominee')).limit(1)\
    .join(nominee_info, on=nominee_info.name == F.col('nominee'), how='inner')\
    .select(F.col('top_genre'))
    
top_winner.show(truncate=False)


+---------+
|top_genre|
+---------+
|Drama    |
+---------+

